In [3]:

import json
import os
import polars as pl
from polars import col as c

from smallflex_data_schema import SmallflexInputSchema
from plotly.subplots import make_subplots

from datetime import date
import plotly.graph_objs as go

from config import settings
import plotly.express as px
from itertools import product
import tqdm

from general_function import pl_to_dict

from data_display.input_data_plots import (
    plot_market_prices, plot_discharge_flow, plot_power, plot_basin_height, plot_basin_height_volume_table,
    plot_continuous_performance_table, plot_discrete_performance_table
)

os.chdir(os.getcwd().replace("/src", ""))


In [ ]:
small_flex_input_schema: SmallflexInputSchema = SmallflexInputSchema()

In [4]:
output_file_names: dict[str, str] = json.load(open(settings.FILE_NAMES))
small_flex_input_schema: SmallflexInputSchema = SmallflexInputSchema().duckdb_to_schema(output_file_names["duckdb_input"])

Read and validate tables from small_flex_input_data.db file: 100%|████████████████████████████████████████████████████| 16/16 [00:04<00:00,  3.75it/s]


In [3]:
def plot_power_performance_table(small_flex_input_schema: SmallflexInputSchema) -> dict:
    plot_dict: dict = dict()
    hydro_power_performance_table: pl.DataFrame = small_flex_input_schema.hydro_power_performance_table
    power_plant_fk = hydro_power_performance_table["power_plant_fk"].unique().to_list()
    
    for metadata in small_flex_input_schema.hydro_power_plant.filter(c("uuid").is_in(power_plant_fk)).to_dicts():
        data = hydro_power_performance_table.filter(c("power_plant_fk")==metadata["uuid"])
        if metadata["control"] == "discrete":
            plot_dict[metadata["name"]] = plot_discrete_performance_table(data)
        elif metadata["control"] == "continuous":
            plot_dict[metadata["name"]] = plot_continuous_performance_table(data)
        
    return plot_dict

In [5]:
plot_dict= plot_power_performance_table(small_flex_input_schema=small_flex_input_schema)

In [ ]:
small_flex_input_schema.hydro_power_plant.filter(c("name").is_in(["Aegina hydro"])).with_row_index(name="H")

H,name,uuid,resource_fk_list,upstream_basin_fk,downstream_basin_fk,rated_power,rated_flow,control,type
u32,str,str,list[str],str,str,f64,f64,str,str
0,"""Aegina hydro""","""b65e48e9-9b05-5dfb-8e98-e61815…","[""f7d845ff-ad6b-58d4-accd-59b1c8cc71b8"", ""db200379-c96b-5685-a446-a9d285801946"", ""b039c287-9763-5aec-b12b-0935cc2c7caf""]","""42d06c8a-be47-55e2-9644-eda169…","""fa0d5019-59c1-5181-81f7-a5fe9f…",9.2,2.8,"""discrete""","""buildup_pump_turbine"""
0,"""Aegina hydro""","""b65e48e9-9b05-5dfb-8e98-e61815…","[""f7d845ff-ad6b-58d4-accd-59b1c8cc71b8"", ""db200379-c96b-5685-a446-a9d285801946"", ""b039c287-9763-5aec-b12b-0935cc2c7caf""]","""42d06c8a-be47-55e2-9644-eda169…","""fa0d5019-59c1-5181-81f7-a5fe9f…",9.2,2.8,"""discrete""","""buildup_pump_turbine"""


In [13]:
plot_dict["Aegina hydro"]

In [12]:
plot_dict["Merezenbach"]

In [11]:
plot_dict["Morel"]